# Connection to Graph Database

In [17]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "password2"

try:
    driver = GraphDatabase.driver(uri, auth=(username, password))
    session = driver.session()
    result = session.run("RETURN 1")
    value = result.single()[0]
    print("Connection successful. Returned value:", value)
except Exception as e:
    print("Connection error:", str(e))

Connection successful. Returned value: 1


# Functions to retrieve information about nodes from graph

Function to get Recipe node with given ingredient list

In [23]:
#function for retrieving recipes with all ingredients
def get_recipes_with_all_ingredients(ingredients_list):
    query = """
    MATCH (r:Recipe)
    WHERE ALL(ingredientName IN $ingredients_list
              WHERE (:Ingredient {name: ingredientName})-[:Ingredient_In]->(r))
    RETURN r.name
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, ingredients_list=ingredients_list)
            return [record["r.name"] for record in result]

Function to get weights of ingredients in given ingredient list 

In [24]:
#function to get weight of each ingredient  in the ingredient list
def get_ingredient_weight(ingredients_list):
    query = """
    MATCH (i:Ingredient)
    WHERE i.name IN $ingredients_list
    RETURN i.name, i.weight
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, ingredients_list=ingredients_list)
            ingre_with_weight = {}
            for record in result:
                ingre_with_weight[record["i.name"]] = record["i.weight"]
            return ingre_with_weight

Function to get rating of given recipe

In [25]:
#function to get rating of a recipe
def get_recipe_rating(recipe_name):
    query = """
    MATCH (r:Recipe {name: $recipe_name})
    RETURN r.rating
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, recipe_name=recipe_name)
            return result.single()[0]


In [36]:
#function to get URL of recipe
def get_recipe_url(recipe_name):
    query = """
    MATCH (r:Recipe {name: $recipe_name})
    RETURN r.url
    """

    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, recipe_name=recipe_name)
            return result.single()[0]
        

In [26]:
#function to get all ingredients in a recipe
def get_ingredients_in_recipe(recipe_name):
    query = """
    MATCH (i:Ingredient)-[:Ingredient_In]->(r:Recipe {name: $recipe_name})
    RETURN i.name
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, recipe_name=recipe_name)
            return [record["i.name"] for record in result]
        


In [29]:
#Function to write cypher query to total weight of ingredients in the recipe
def get_total_weight_of_ingredients(ingredients_list):
    query = """
    MATCH (i:Ingredient)
    WHERE i.name IN $ingredients_list
    RETURN sum(i.weight)
    """
    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, ingredients_list=ingredients_list)
            return result.single()[0]
    

In [32]:
ingre = get_ingredients_in_recipe("Masala Karela Recipe")
final_idf=get_total_weight_of_ingredients(ingre)/len(ingre)

# Functions to retrive and calculate recipes score based on idf weighting, fraction of ingredients, nutrients, ratings

Functions to get combination set of ingredients with lower idf weight

In [6]:
import itertools
def get_combinations_525(list1):
    i = len(list1) * 0.525
    list1_525_combinations = []
    list1_525_combinations.extend(list(itertools.combinations(list1, round(i))))
    return list1_525_combinations

def get_combinations_35(list1):
    i = len(list1) * 0.35
    list1_35_combinations = []
    list1_35_combinations.extend(list(itertools.combinations(list1, round(i))))
    return list1_35_combinations

Function to 75%  and 50% of the ingredients in the list 

In [7]:
#combinations of a list elements with 30% of elements with highest weight in the list , other 70% of elements should be choosen randomly and total 75% of elements of original list should be there in new list
def get_75_percent_of_lists(list1):
    ingre_weight = get_ingredient_weight(list1)
    ingre_weight = sorted(ingre_weight.items(), key=lambda x: x[1], reverse=True)
    ingre_list = [x[0] for x in ingre_weight]
    final_list = []
    #get 30% of elements with highest weight
    list1_30 = ingre_list[:int(len(ingre_list)*0.225)]
    list1_not_30 = ingre_list[int(len(ingre_list)*0.225):]
    #get 70% of elements with lower weight forming diffirent combinations and new list should have 75% of elements of original list
    list1_70s = get_combinations_525(list1_not_30)
    for i in list1_70s:
        final_list.append(list1_30 + list(i))

    return final_list


def get_50_percent_of_lists(list1):
    ingre_weight = get_ingredient_weight(list1)
    ingre_weight = sorted(ingre_weight.items(), key=lambda x: x[1], reverse=True)
    ingre_list = [x[0] for x in ingre_weight]
    final_list = []
    #get 30% of elements with highest weight
    list1_30 = ingre_list[:int(len(ingre_list)*0.15)]
    list1_not_30 = ingre_list[int(len(ingre_list)*0.15):]
    #get 70% of elements with lower weight forming diffirent combinations and new list should have 75% of elements of original list
    list1_70s = get_combinations_35(list1_not_30)
    for i in list1_70s:
        final_list.append(list1_30 + list(i))

    return final_list

Function to retrieve recipes with all ingredients and 75% of ingredients and 50% of ingredients

In [31]:
#function for retrieving recipes with all possible combinations of ingredients
def get_recipes_with_ingredients_combinations_v2(ingredients_list):
    total_recipe_list = []
    total_recipe_dict = dict()
    result1 = get_recipes_with_all_ingredients(ingredients_list)
    total_recipe_list.extend(result1)
    for recipe in result1:
        total_recipe_dict[recipe] = 0.5

    combinations_75 = get_75_percent_of_lists(ingredients_list)
    for combination in combinations_75:
        result2 = get_recipes_with_all_ingredients(combination)
        for recipe in result2:
            if recipe not in total_recipe_dict:
                total_recipe_dict[recipe] = 0.375
                total_recipe_list.append(recipe)

    combinations_50 = get_50_percent_of_lists(ingredients_list)
    for combination in combinations_50:
        result3 = get_recipes_with_all_ingredients(combination)
        for recipe in result3:
            if recipe not in total_recipe_dict:
                total_recipe_dict[recipe] = 0.25
                total_recipe_list.append(recipe)

    return total_recipe_list, total_recipe_dict

Function to get nutrients of given recipe

In [8]:
#get nutrients of recipe 
def get_nutrients(recipe_name):
    query = """
    MATCH (r:Recipe {name: $recipe_name})
    RETURN r.nutrients
    """

    with driver.session() as session:
        with session.begin_transaction() as tx:
            result = tx.run(query, recipe_name=recipe_name)
            return result.single()[0]
        

Function to extract nutrients name

In [9]:
#function to extract until alphabet
def extract_until_alphabet(s):
    extracted = ""
    for char in s:
        if char.isalpha():
            break
        extracted += char
    return float(extracted)

Functions to calculate score or retrieved recipes based on one of three diet criterions

In [10]:
weight_gain_weights = {
    'calories': 0.25,
    'total_fat': 0.15,
    'saturated_fat': 0.1,
    'fiber': 0.05,
    'carbohydrate': 0.2,
    'protein': 0.2,
    'sugar': 0.05,
}
weight_loss_weights = {
    'calories': 0.1,
    'total_fat': 0.1,
    'saturated_fat': 0.05,
    'fiber': 0.25,
    'carbohydrate': 0.2,
    'protein': 0.2,
    'sugar': 0.1,
}
balanced_diet_weights = {
    'calories': 0.15,
    'total_fat': 0.15,
    'saturated_fat': 0.1,
    'fiber': 0.15,
    'carbohydrate': 0.2,
    'protein': 0.2,
    'sugar': 0.05,
}
import ast
# calculate the gain weight max score from all the recipes
maxb=1.5142857142857142
maxg=2.378571428571429
def weight_gain(nutrients):
    score=0.0
    # print(df1['Nutrients'][index]['calories'])
    try:
        for key,value in weight_gain_weights.items():
            # print(key)
            score+=value*(extract_until_alphabet(ast.literal_eval(nutrients)[key]))
        score/=extract_until_alphabet(ast.literal_eval(nutrients)['serving_size'])
        score=score/maxg
        return score
        # if score>max:
        #     max=score
        #     recipe=data
    except:
        pass
    
def weight_loss(nutrients):
    
    score=0.0
# print(df1['Nutrients'][index]['calories'])
    try:
        for key,value in weight_loss_weights.items():
            # print(key)
            score+=value*(extract_until_alphabet(ast.literal_eval(nutrients)[key]))
        score/=extract_until_alphabet(ast.literal_eval(nutrients)['serving_size'])
        return score
        # print(score)
        # if score>max:
        #     max=score
        #     recipe=data
    except:
        pass
def balanced_diet(nutrients):
   
    score=0.0
# print(df1['Nutrients'][index]['calories'])
    try:
        for key,value in balanced_diet_weights.items():
            # print(key)
            score+=value*(extract_until_alphabet(ast.literal_eval(nutrients)[key]))
        score/=extract_until_alphabet(ast.literal_eval(nutrients)['serving_size'])
        score=score/maxb
        return score
        # print(score)
        # if score>max:
        #     max=score
        #     recipe=data
    except:
        pass

In [13]:
#function to stem the list of words
from nltk.stem import PorterStemmer
def stem_list(list1):
    ps = PorterStemmer()
    stemmed_list = []
    for i in list1:
        stemmed_list.append(ps.stem(i))
    return stemmed_list



In [37]:
import math
def RRS(ingredient_list, diet_type):
    #stem the ingreidients
    ingredient_list = stem_list(ingredient_list)
    #get recipes with all possible combinations of ingredients
    retreived_recipes_list, retreived_recipes_dict  = get_recipes_with_ingredients_combinations_v2(ingredient_list)

    #calculate score for each recipe on the basis of rating, diet type
    for recipe in retreived_recipes_dict:
        rating = get_recipe_rating(recipe)
        if math.isnan(rating):
            rating = 0.0
        new_value = (rating/5) * 0.25 
        
        nutrients = get_nutrients(recipe)
        if diet_type == "balanced_diet":
            n_score = balanced_diet(nutrients)
        elif diet_type == "weight_loss":
            n_score = weight_loss(nutrients)
        elif diet_type == "weight_gain":
            n_score = weight_gain(nutrients)
        else:
            n_score = None

        if n_score is None:
            n_score = 0.0

        final_score = n_score * 0.25 + retreived_recipes_dict[recipe] + new_value 
        retreived_recipes_dict[recipe] = final_score

    #sort the recipes on the basis of score 
    sorted_retrieved_recipes_dict = dict(sorted(retreived_recipes_dict.items(), key=lambda item:item[1], reverse=True ))
    #get top 5 recipes in sorted_retrieved_recipes_dict
    recipes_dict = dict.fromkeys(list(sorted_retrieved_recipes_dict.keys())[:5], "")
    #get url of each recipe
    for recipe in recipes_dict:
        recipes_dict[recipe] = get_recipe_url(recipe)

    return recipes_dict

**************************************************

# RESULT

In [39]:
ingredients_list1 = ["makki ka atta", "prawn", "red chilli flak", "corainder leav", "curd", "lemon ju"]
#diet type = 1 for weight gain, 2 for weight loss, 3 for balanced diet
diet_type = 1
result = RRS(ingredients_list1, diet_type)
i = 0
for recipe in result:
    print(i+1,recipe, result[recipe])
    i+=1

1 Coriander Prawn Cakes Recipe https://www.archanaskitchen.com/coriander-prawn-cakes-recipe
2 Spinach Soft Shell Taco Stuffed with Cabbage Slaw Salad Recipe https://www.archanaskitchen.com/spinach-soft-shell-taco-stuffed-with-cabbage-slaw-salad-recipe
3 Savory Ricotta Tart Recipe https://www.archanaskitchen.com/savory-ricotta-tart-recipe
4 Chicken Crepe Lasagne Recipe http://www.archanaskitchen.com/chicken-crepe-lasagne-recipe
5 Andaman Style Steamed Garlic Prawns Recipe https://www.archanaskitchen.com/andaman-style-steamed-garlic-prawns-recipe


In [42]:
#
ingredients_list2 = ["paneer", "potato", "cumin powder","green bell pepper", "whole cashew", "green peas", "butter", "lemon juice"]
#diet type = 1 for weight gain, 2 for weight loss, 3 for balanced diet
diet_type = 1
result = RRS(ingredients_list2, diet_type)
i = 0
for recipe in result:
    print(i+1,recipe, result[recipe])
    i+=1

1 Dahi Wale Paneer Recipe - Paneer In A Curd Gravy https://www.archanaskitchen.com/dahi-wale-paneer-recipe-paneer-in-a-curd-gravy
2 Paneer Peshawari Recipe - Cottage Cheese In Rich Gravy http://www.archanaskitchen.com/paneer-peshawari-recipe-cottage-cheese-in-rich-gravy
3 Achari Paneer Masala Recipe http://www.archanaskitchen.com/achari-paneer-masala-recipe
4 Nawabi Paneer Curry Recipe https://www.archanaskitchen.com/nawabi-paneer-curry-recipe
5 Matar Paneer Kachori Korma Recipe - Green Peas And Paneer Kachori Curry http://www.archanaskitchen.com/matar-paneer-kachori-korma-recipe-green-peas-and-paneer-kachori-curry


In [46]:
ingredients_list3 = ['garam masala powd', 'fresh cream', 'butter', 'cashew nut', 'gram pan', 'cumin powd', 'kasuri methi']
#diet type = 1 for weight gain, 2 for weight loss, 3 for balanced diet
diet_type = 3
result = RRS(ingredients_list2, diet_type)
i = 0
for recipe in result:
    print(i+1,recipe, result[recipe])
    i+=1

1 Dahi Wale Paneer Recipe - Paneer In A Curd Gravy https://www.archanaskitchen.com/dahi-wale-paneer-recipe-paneer-in-a-curd-gravy
2 Paneer Peshawari Recipe - Cottage Cheese In Rich Gravy http://www.archanaskitchen.com/paneer-peshawari-recipe-cottage-cheese-in-rich-gravy
3 Achari Paneer Masala Recipe http://www.archanaskitchen.com/achari-paneer-masala-recipe
4 Nawabi Paneer Curry Recipe https://www.archanaskitchen.com/nawabi-paneer-curry-recipe
5 Matar Paneer Kachori Korma Recipe - Green Peas And Paneer Kachori Curry http://www.archanaskitchen.com/matar-paneer-kachori-korma-recipe-green-peas-and-paneer-kachori-curry
